In [ ]:
from enum import Enum
from random import shuffle
import numpy as np
from operator import mul
import random
import termcolor
import pandas as pd

class Suit(Enum):
    SPADE = '♠'
    CLUB = '♣'
    HEART = '♡'
    DIAMOND = '♢'
    def __str__(self):
        return self.value
    def __repr__(self):
        return f"Suit.{self.name}"


class Number(Enum):
    ACE = (1, 'A')
    TWO = (2, '2')
    THREE = (3, '3')
    FOUR = (4, '4')
    FIVE = (5, '5')
    SIX = (6, '6')
    SEVEN = (7, '7')
    EIGHT = (8, '8')
    NINE = (9, '9')
    TEN = (10, '10')
    JACK = (11, 'J')
    QUEEN = (12, 'Q')
    KING = (13, 'K')

    def __init__(self, val, string):
        self.val = val
        self.string = string

    def __str__(self):
        return self.string

    def __repr__(self):
        return f"Number.{self.name}"
    
class Card:
    def __init__(self, suit, number):
        if not (isinstance(suit, Suit) and isinstance(number, Number)):
            raise ValueError  # Enum じゃないとエラー
        self.suit = suit
        self.number = number

    def __str__(self):
        return str(self.suit) + str(self.number)

    def __repr__(self):
        return f"Card({self.__str__()})"
    
    def __eq__(self, other):
        return (self.suit, self.number) == (other.suit, other.number)
            
    
class Hand(list):
    def __init__(self,card_list):
        super().__init__(
            i for i in card_list
        )
        
    def check_number(self):
        number_list=[i.number.val for i in self]
        return number_list
    
    def check_suit(self):
        suit_list=[str(i.suit) for i in self]
        return suit_list
    
    def choice(self,card):
        #Card(Suit.SPADE, Number.ACE)
        if card in self:
            self.remove(card)
            return card
        else:
            raise ValueError
            
    def check(self,card):
        return card in self
    

class Deck(list):
    def __init__(self):
        super().__init__(
            Card(suit, number) for suit in Suit for number in Number
        )  # list の初期化を呼び出す
        self.shuffle()  # 最初にシャッフル
    def shuffle(self):
        shuffle(self)
    def draw(self):
        return self.pop()
    def deal(self, players_num):
        cards=[Hand(i) for i in np.array_split(self,players_num)]
        self.clear()
        return cards

In [ ]:
# ゲームの状態
class State:
    # 初期化
    def __init__(self, players_num=3,field_cards=None, players_cards=None,turn_player=None,pass_count=None,out_player=None,all_actions=None,all_players=None):
        if players_cards==None:
            deck = Deck()
            self.players_cards=deck.deal(players_num)
            self.players_num=players_num
            self.field_cards=np.zeros((4,13), dtype='int64')
            self.start_flags=[0]*self.players_num
            self.pass_count=[0]*self.players_num
            self.out_player=[]
            self.all_actions = []
            self.all_players = []
            self.all_cards=[[str(Card(suit, number))  for number in Number] for suit in Suit]
            for players_number in range(players_num):
                self.start_flags[players_number]=self.choice_seven(hand=self.players_cards[players_number])
            self.turn_player=self.start_flags.index(1)
        else:
            self.players_cards=players_cards
            self.field_cards=field_cards
            self.players_num=players_num
            self.turn_player=turn_player
            self.pass_count=pass_count
            self.out_player=out_player
            self.all_actions=all_actions
            self.all_players =all_players
            self.all_cards=[[str(Card(suit, number))  for number in Number] for suit in Suit]
        

    #7のカードを出す
    def choice_seven(self,hand):
        start_flag=0
        for card in [Card(suit,Number.SEVEN) for suit in Suit]:
            if hand.check(Card(Suit.DIAMOND,Number.SEVEN))==True:
                start_flag=1
            if hand.check(card)==True:
                self.put_card(hand.choice(card))
        return start_flag
    
    def choice_card(self,hand,card):
        hand.choice(card)
    
    #場にカードを出す
    def put_card(self,card):
        num=10
        for s,i in zip(Suit,range(4)):
            if card.suit==s:
                num=i
        #state.my_hands().remove(card)
        self.field_cards[num][card.number.val-1]=int(1)
    
    
    # 場で出せる手のリスト取得
    # 3パスの人がいた時、未対応
    def legal_actions(self):
        actions = []
        for suit,n in zip(Suit,range(4)):
            
            if self.field_cards[n][0:6][::-1].tolist().count(1)!=6:
                actions.append(Card(suit,self.num_to_Enum(6-self.field_cards[n][0:6][::-1].tolist().index(0))))
                
            if self.field_cards[n][7:13].tolist().count(1)!=6:
                actions.append(Card(suit,self.num_to_Enum(8+self.field_cards[n][7:13].tolist().index(0))))
        return actions
    
    
    # 自分が出せる手のリスト取得
    def my_actions(self):
        actions = []
        for legal in self.legal_actions():
            if self.players_cards[self.turn_player].check(legal)==True:
                actions.append(legal)
        return actions
    def my_actions_str(self):
        actions = []
        for legal in self.legal_actions():
            if self.players_cards[self.turn_player].check(legal)==True:
                actions.append(legal)
        return [str(i) for i in actions]
    
    # 自分の手札取得
    def my_hands(self):
        return self.players_cards[self.turn_player]
    def my_hands_str(self):
        return [str(i) for i in self.players_cards[self.turn_player]]
            
    
    def num_to_Enum(self,num):
        enum_list=[Number.ACE,Number.TWO,Number.THREE,Number.FOUR, 
                   Number.FIVE,Number.SIX,Number.SEVEN,Number.EIGHT,
                   Number.NINE,Number.TEN,Number.JACK,Number.QUEEN,
                   Number.KING]
        return enum_list[num-1]
    
    
    # 次の状態の取得
    def next(self, action,pass_flag = 0):
        if self.my_actions()==[]:
            self.pass_count[self.turn_player]+=1
            self.pass_check()
        elif pass_flag == 1:
            self.pass_count[self.turn_player]+=1
            self.pass_check()
        else:
            self.players_cards[self.turn_player].remove(action)
            self.put_card(action)
            self.all_actions.append(action)
            self.all_players.append(self.turn_player)
            
        #次のプレイヤーに
        self.next_player() 
        return State(players_num=self.players_num,field_cards=self.field_cards, players_cards=self.players_cards,turn_player=self.turn_player,pass_count=self.pass_count,out_player=self.out_player,all_actions=self.all_actions,all_players=self.all_players)
   
    #次のプレイヤーの取得 
    def next_player(self):
        flag=0
        while flag==0:
            if self.turn_player+1>=self.players_num:
                self.turn_player=self.turn_player+1-self.players_num
            else:
                self.turn_player+=1

            if self.turn_player not in self.out_player:
                flag=1
    
    #パスの上限判定
    def pass_check(self):
        out_list=self.out_player
        if self.pass_count[self.turn_player]>3:
            for card in self.my_hands():
                self.put_card(card)
                self.all_actions.append(card)
                self.all_players.append(self.turn_player)
                
            out_list.append(self.turn_player)
            
            self.out_player=out_list
            
    def to_str(self,num):
        return str(num)
    
    #勝ち負け判定
    def is_done(self):
        return len(self.my_hands())==0
        
            
    # 状態表示
    def __str__(self):
        str = ''
        field_cards=self.field_cards.tolist()
        out_list=[list(map(mul,self.all_cards[i],field_cards[i])) for i in range(4)]
        str += "場のカード\n\n"
        for i in range(len(out_list)):
            minilist=out_list[i]
            for j in range(len(minilist)):
                if minilist[j] == "":
                    str += " -- "
                else:
                    str +=" "+minilist[j]+" "
            str += '\n'
        num=self.to_str(self.turn_player)
        pass_cnt=self.to_str(self.pass_count[self.turn_player])
        str+="\nプレイヤー"+num+"番　　パス回数"+pass_cnt+"\n"
        str += "\nあなたの手札\n"
        
        out_list=self.my_hands_str()
        for i in range(len(out_list)):
            str+=out_list[i]
            str+=" "
            
        str += "\n\n出せるカード\n"
        
        out_list=self.my_actions_str()
        for i in range(len(out_list)):
            str+=out_list[i]
            str+=" "
        
        str += "\n"
        
        return str    

In [ ]:
def num_to_Card(number,suit):
    number_list=[Number.ACE,Number.TWO,Number.THREE,Number.FOUR, 
                Number.FIVE,Number.SIX,Number.SEVEN,Number.EIGHT,
                Number.NINE,Number.TEN,Number.JACK,Number.QUEEN,
                Number.KING]
    suit_list=[Suit.SPADE,Suit.CLUB,Suit.HEART,Suit.DIAMOND]
    return Card(suit_list[suit],number_list[number-1])

In [ ]:
# ランダム行動 AI
def random_action(state):
    my_actions = state.my_actions()
    if my_actions != []:
        return my_actions[random.randint(0, len(my_actions)-1)]
    else:
        my_actions=[]
        return my_actions

In [ ]:
#ライブラリを追加で使用したい場合はここでimportして下さい。

MY_PLAYER_NUM = 0

#自分のAI
def my_AI(state):
    #パスしない時　pass_flag = 0
    #出せるけどパスしたい時　pass_flag = 1
    #3回以上パスしてしまうとアウトになってしまいます。
    pass_flag = 0
    
    return random_action(state),pass_flag

In [ ]:
# ランダムAIと対戦
state = State()
# ゲーム終了までのループ
while True:
    # ゲーム終了時
    if state.is_done():
        print("勝者 プレイヤー"+str(state.turn_player)+"番")
        break;
        
    pass_flag = 0
    # 行動の取得
    if state.turn_player == MY_PLAYER_NUM:
        action,pass_flag = my_AI(state)
        print(termcolor.colored(state, 'red'))
    else:
        action = random_action(state)
        print(state)

    # 次の状態の取得
    if pass_flag == 1:
        state = state.next(action,pass_flag)
    else:
        state = state.next(action)

場のカード

 --  --  --  --  --  --  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー1番　　パス回数0

あなたの手札
♣6 ♡5 ♡8 ♠5 ♡J ♣5 ♠10 ♣J ♣K ♢9 ♡3 ♡K ♢3 

出せるカード
♣6 ♡8 

場のカード

 --  --  --  --  --  --  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  ♡8  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー2番　　パス回数0

あなたの手札
♡4 ♣Q ♠Q ♠A ♢5 ♠3 ♠K ♣A ♡2 ♢A ♡10 ♢J ♠J ♢6 ♢4 ♠6 ♢2 

出せるカード
♠6 ♢6 

場のカード

 --  --  --  --  --  ♠6  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  ♡8  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー0番　　パス回数0

あなたの手札
♠2 ♡Q ♣10 ♡A ♡9 ♣3 ♠9 ♡6 ♣8 ♣4 ♣9 ♢Q ♠4 ♣2 ♢8 ♠8 ♢K ♢10 

出せるカード
♠8 ♣8 ♡6 ♡9 ♢8 

場のカード

 --  --  --  --  --  ♠6  ♠7  --  --  --  --  --  -- 
 --

In [ ]:
# ランダムAIとの勝率チェック
# パラメータ
EP_GAME_COUNT = 2000  # 1評価あたりのゲーム数

def player_point(ended_state):
    #print(termcolor.colored(ended_state.turn_player, 'red'))
    if ended_state.turn_player==0:
        return 1
    return 0

def play(next_actions):
    state = State()
    while True:
        if state.is_done():
            break
        pass_flag = 0
        if state.turn_player==MY_PLAYER_NUM:
            action,pass_flag = my_AI(state)
        else:
            action = random_action(state)
        # 次の状態の取得
        if pass_flag == 1:
            state = state.next(action,pass_flag)
        else:
            state = state.next(action)     
    return player_point(state)

# 任意のアルゴリズムの評価
def evaluate_algorithm_of(label, next_actions):
    # 複数回の対戦を繰り返す
    total_point = 0
    for i in range(EP_GAME_COUNT):
        total_point += play(next_actions)
        print('\rEvaluate {}/{}'.format(i + 1, EP_GAME_COUNT), end='')
    print('')

    # 平均ポイントの計算
    average_point = total_point / EP_GAME_COUNT
    print(label.format(average_point))

# VSランダム
next_actions = (random_action, random_action)
evaluate_algorithm_of('VS_Random {:.3f}', next_actions)

Evaluate 2000/2000
VS_Random 0.276


## 決勝大会用にいくつかの追加の機能が実装されています。

### カードが出せるときのPASS
カードを持っているけど、戦略的にpassをしたい時用です。

In [ ]:
state = State()

In [ ]:
state = state.next(random_action(state),pass_flag=1)

In [ ]:
print(state)

場のカード

 --  --  --  --  --  --  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー3番　　パス回数0

あなたの手札
♢J ♣5 ♠A ♡K ♢A ♠Q ♡8 ♢2 ♡Q ♠K ♣2 ♠4 ♣3 

出せるカード
♡8 



In [ ]:
#パス回数のカウント
state.pass_count

[0, 0, 1, 0]

In [ ]:
#今のプレイヤーの番号
state.turn_player

3

In [ ]:
#今のプレイヤーのパス回数
state.pass_count[state.turn_player]

0

### 今まで出たカードと、そのカードを出したプレイヤー
対戦相手の手札がどのような物か推測できる可能性があります

In [ ]:
#今までのデータカード
state.all_actions

[Card(♡8),
 Card(♠6),
 Card(♢8),
 Card(♣6),
 Card(♢6),
 Card(♡9),
 Card(♠5),
 Card(♢5),
 Card(♡6),
 Card(♣5),
 Card(♢4),
 Card(♣4),
 Card(♡5),
 Card(♡4),
 Card(♣3),
 Card(♢3),
 Card(♢2),
 Card(♠8),
 Card(♡3),
 Card(♡10),
 Card(♠9),
 Card(♡J),
 Card(♡2),
 Card(♡Q),
 Card(♢9),
 Card(♢A),
 Card(♡A),
 Card(♡K),
 Card(♣2),
 Card(♠10),
 Card(♣A),
 Card(♣8),
 Card(♠J),
 Card(♣9),
 Card(♠Q),
 Card(♠4),
 Card(♠3),
 Card(♢10),
 Card(♣J),
 Card(♣K),
 Card(♢J),
 Card(♢Q),
 Card(♠K),
 Card(♢K),
 Card(♣10),
 Card(♣Q),
 Card(♠2),
 Card(♠A)]

In [ ]:
#それを出したプレイヤー
state.all_players

[1,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 1,
 1,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2]

In [ ]:
def cal_player_num(n):
    out_num = MY_PLAYER_NUM + n
    if out_num > 3:
        out_num = out_num-3
    if out_num < 0:
        out_num = out_num+3
    return out_num

In [ ]:
cal_player_num(0)

0

In [ ]:
c.suit == Suit.SPADE

True

In [ ]:
count_h_before = 0
for c,p in zip(state.all_actions,state.all_players):
    if p == cal_player_num(-1):
         if c.suit == Suit.HEART:
            count_h_before +=1

In [ ]:
count_h_before

3

## 自分の出せるカードから特定のマークのものを探す

In [ ]:
state = State()

In [ ]:
state.my_actions()

[Card(♠8), Card(♣8), Card(♡8), Card(♢8)]

In [ ]:
my_a = state.my_actions()

In [ ]:
[a.suit for a in my_a]

[Suit.SPADE, Suit.CLUB, Suit.HEART, Suit.DIAMOND]

In [ ]:
suit = [a.suit for a in my_a]

In [ ]:
idx = suit.index(Suit.CLUB) 
idx

1

In [ ]:
state.my_actions()[idx]

Card(♣8)

## 手札を並び替える(復習)

In [ ]:
state.my_hands().check_number()

[13, 13, 2, 1, 8, 2, 5, 10, 8, 1, 1, 3, 4, 8, 8, 11]

In [ ]:
def DistFrom7(hand):
    return abs(hand.number.val)

hands_sorted = sorted(state.my_hands(), key = DistFrom7, reverse = False)
print(state.my_hands())
print(hands_sorted)

[Card(♡K), Card(♣K), Card(♠2), Card(♣A), Card(♢8), Card(♣2), Card(♠5), Card(♢10), Card(♡8), Card(♢A), Card(♠A), Card(♡3), Card(♡4), Card(♠8), Card(♣8), Card(♢J)]
[Card(♣A), Card(♢A), Card(♠A), Card(♠2), Card(♣2), Card(♡3), Card(♡4), Card(♠5), Card(♢8), Card(♡8), Card(♠8), Card(♣8), Card(♢10), Card(♢J), Card(♡K), Card(♣K)]


In [ ]:
state=State()

In [ ]:
#自分が出せるカードを並び替える場合

action_sorted = sorted(Hand(state.my_actions()), key = DistFrom7, reverse = False)
print(state.my_actions())
print(action_sorted)

[Card(♣6), Card(♣8), Card(♡8)]
[Card(♣6), Card(♣8), Card(♡8)]


## 評価値の検討

In [ ]:
# ランダムAIと対戦
state = State()
# 20ターン進める
for ppp in range(20):
    
    # ゲーム終了時
    if state.is_done():
        print("勝者 プレイヤー"+str(state.turn_player)+"番")
        break;
        
    pass_flag = 0
    # 行動の取得
    if state.turn_player == MY_PLAYER_NUM:
        action,pass_flag = my_AI(state)
        print(termcolor.colored(state, 'red'))
    else:
        action = random_action(state)
        print(state)

    # 次の状態の取得
    if pass_flag == 1:
        state = state.next(action,pass_flag)
    else:
        state = state.next(action)

場のカード

 --  --  --  --  --  --  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー0番　　パス回数0

あなたの手札
♢8 ♡A ♠6 ♢A ♠A ♡J ♡K ♣5 ♢Q ♡8 ♢K ♣A ♣Q ♠8 ♣2 ♠5 

出せるカード
♠6 ♠8 ♡8 ♢8 

場のカード

 --  --  --  --  --  ♠6  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー1番　　パス回数0

あなたの手札
♠10 ♡3 ♢5 ♡2 ♢J ♠4 ♢4 ♣6 ♡Q ♠3 ♣3 ♠2 ♣K ♠J ♣4 

出せるカード
♣6 

場のカード

 --  --  --  --  --  ♠6  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  ♣6  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー2番　　パス回数0

あなたの手札
♠K ♣8 ♣J ♡10 ♢3 ♡6 ♠Q ♢9 ♢10 ♢6 ♠9 ♡5 ♡9 ♢2 ♣9 ♡4 ♣10 

出せるカード
♣8 ♡6 ♢6 

場のカード

 --  --  --  --  --  ♠6  ♠7  --  --  --  --  --  -- 
 --  -

In [ ]:
print(state)

場のカード

 --  --  ♠3  ♠4  ♠5  ♠6  ♠7  ♠8  ♠9  ♠10  ♠J  --  -- 
 --  --  --  --  ♣5  ♣6  ♣7  --  --  --  --  --  -- 
 --  --  --  --  ♡5  ♡6  ♡7  ♡8  ♡9  --  --  --  -- 
 --  --  ♢3  ♢4  ♢5  ♢6  ♢7  ♢8  --  --  --  --  -- 

プレイヤー2番　　パス回数0

あなたの手札
♠K ♣8 ♣J ♡10 ♠Q ♢9 ♢10 ♢2 ♣9 ♡4 ♣10 

出せるカード
♠Q ♣8 ♡4 ♡10 ♢2 ♢9 



In [ ]:
state.field_cards

array([[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])

In [ ]:
state.field_cards[0].sum()

9

In [ ]:
state.field_cards[1].sum()

3

In [ ]:
temp = np.array([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]])

In [ ]:
up = state.field_cards*temp
state.field_cards*temp

array([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [ ]:
up[0].sum()

4

### 1.手札に関する関数

In [ ]:
 #リストで手札を表示する　
state.my_hands()

[Card(♡3),
 Card(♣5),
 Card(♠8),
 Card(♣4),
 Card(♣8),
 Card(♡2),
 Card(♣10),
 Card(♡J),
 Card(♠J),
 Card(♠4),
 Card(♠A),
 Card(♡10)]

In [ ]:
#リストで手札の数字を表示する
state.my_hands().check_number()

[3, 5, 8, 4, 8, 2, 10, 11, 11, 4, 1, 10]

In [ ]:
#リストで手札のマークを表示する
state.my_hands().check_suit()

['♡', '♣', '♠', '♣', '♣', '♡', '♣', '♡', '♠', '♠', '♠', '♡']

In [ ]:
#リストで自分が出せるカードを表示する
state.my_actions()

[Card(♠8), Card(♣8)]

In [ ]:
#リストで自分が出せるカードの数字を表示する
Hand(state.my_actions()).check_number()

[8, 8]

In [ ]:
#リストで自分が出せるカードの記号を表示する
Hand(state.my_actions()).check_suit()

['♠', '♣']

### 2.場の札に関する関数

In [ ]:
#場のカードを表示する
state.field_cards

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])

In [ ]:
state.field_cards[0,6]

1

In [ ]:
#場で出せるカードをリストで取得する
state.legal_actions()

[Card(♠6),
 Card(♠8),
 Card(♣6),
 Card(♣8),
 Card(♡6),
 Card(♡8),
 Card(♢6),
 Card(♢8)]

In [ ]:
#場で出せるカードの数字をリストで取得する
Hand(state.legal_actions()).check_number()

[6, 8, 6, 8, 6, 8, 6, 8]

In [ ]:
#場で出せるカードの記号をリストで取得する
Hand(state.legal_actions()).check_suit()

['♠', '♠', '♣', '♣', '♡', '♡', '♢', '♢']

 ### 3.状態に関する関数

In [ ]:
#今のプレイヤーの番号を表示する
state.turn_player

0

In [ ]:
#3回パスをしてしまったプレイヤーを表示する
state.out_player

[]